<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/KG_Enhanced_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem Statement

#### Task
Develop a co-pilot for threat researchers, security analysts, and professionals that addresses the limitations of current AI solutions like ChatGPT and Perplexity.

#### Current Challenges
1. **Generic Data**: Existing AI solutions provide generic information that lacks specificity.
2. **Context Understanding**: These solutions fail to understand and maintain context.
3. **Limited Information**: The data sources are often limited and not comprehensive.
4. **Single Source Dependency**: Relying on a single source of information reduces reliability and accuracy.
5. **Inadequate AI Models**: Current models do not meet the specialized needs of cybersecurity professionals.

#### Requirement
Create a chatbot capable of collecting and curating data from multiple sources, starting with search engines, and expanding to website crawling and Twitter scraping.

#### Features Required

##### User Interface (UI)
- Chat UI with file upload capabilities.
- Options to save and select prompts.
- Configuration settings for connectors with enable/disable toggles.
- Interface for configuring knowledge and variables (similar to Dify.ai).

##### Technical Specifications
- **No Hallucinations**: Ensure the chatbot provides accurate and reliable information.
- **RAG (Retrieval-Augmented Generation)**: Use RAG to determine which connectors to use based on user inputs.
- **Query Chunking and Distribution**: Optimize the process of breaking down queries and distributing them across different sources.
- **Data Curation Steps**:
  1. Collect links from approximately 50 sources.
  2. Aggregate data from websites and Twitter.
  3. Curate data using a knowledge graph to find relationships and generate responses.
- **Chatbot Capabilities**: Answer queries such as:
  - "List all details on {{BFSI}} security incidents in {{India}}."
  - "List all ransomware attacks targeting the healthcare industry in {{last 7 days/last 3 months/last week/last month}}."
  - "Provide recent incidents related to Lockbit Ransomware gang / BlackBasta Ransomware."

#### Goal
Develop a data collector that integrates multiple specific sources to enrich the knowledge base, enabling the model to better understand context and deliver accurate results. The solution should be modular, allowing customization and configuration of sources.

#### Summary
The goal is to build an advanced, modular chatbot for cybersecurity professionals that overcomes the limitations of existing AI solutions by integrating multiple data sources and ensuring context-aware, accurate responses. The chatbot will utilize state-of-the-art techniques like RAG and knowledge graphs to provide comprehensive, curated information from diverse sources.


#### Installation and Setup

In [1]:
!pip uninstall -yq torch torchvision pandas
!pip install -q torch==2.3.1 torchvision==0.18.1 pandas==2.0.3
!pip install -qU langchain langchain-community faiss-cpu kuzu pyvis
!pip install -qU sentence-transformers networkx pydantic
!pip install -qU langchain-groq apify_client langgraph python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6

**Imports**

In [2]:
import os
import logging
import re
from typing import List, Dict, Any, Optional
from datetime import datetime, timedelta
import requests

import networkx as nx
from pyvis.network import Network
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema import SystemMessage, HumanMessage
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.tools import BaseTool
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.messages import BaseMessage
from langchain.output_parsers import PydanticOutputParser
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field

from apify_client import ApifyClient
from transformers import pipeline

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

**Embedding and LLM Initialization**

In [3]:
# Initialize HuggingFace embeddings
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

# Initialize Llama-3.1 from Meta using Groq LPU Inference
llm = ChatGroq(
    temperature=0,
    model="llama-3.1-70b-versatile",
    api_key="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"
)

# Define system and human messages
system_message = """You are an expert cybersecurity analyst with extensive knowledge in threat analysis,
vulnerability assessment, and security recommendations. Provide detailed, precise, and actionable insights.
Always consider the latest threat intelligence and best practices in your analysis."""
prompt_template = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{text}")])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Knowledge Graph Implementation**

In [4]:
class KnowledgeGraph:
    def __init__(self):
        self.graph = nx.Graph()

    def add_entity(self, entity: str, entity_type: str):
        self.graph.add_node(entity, type=entity_type)
        logger.info(f"Added entity: {entity} of type: {entity_type}")

    def add_relation(self, entity1: str, entity2: str, relation: str):
        self.graph.add_edge(entity1, entity2, relation=relation)
        logger.info(f"Added relation: {relation} between {entity1} and {entity2}")

    def get_related_entities(self, entity: str) -> List[Dict[str, str]]:
        related = [{"entity": neighbor, "relation": self.graph.get_edge_data(entity, neighbor)["relation"]}
                   for neighbor in self.graph.neighbors(entity)]
        logger.info(f"Related entities for {entity}: {related}")
        return related

    def visualize(self, output_file: str = "knowledge_graph.html"):
        net = Network(notebook=True, width="100%", height="500px")
        for node, node_data in self.graph.nodes(data=True):
            net.add_node(node, label=node, title=f"Type: {node_data['type']}")
        for edge in self.graph.edges(data=True):
            net.add_edge(edge[0], edge[1], title=edge[2]['relation'])
        net.show(output_file)
        logger.info(f"Knowledge graph visualized at {output_file}")

# Initialize knowledge graph
kg = KnowledgeGraph()
kg.visualize()

knowledge_graph.html


**Data Collection Functions**

In [5]:
apify_client = ApifyClient("apify_api_yUkcz99gMX1pwNckRi7EyXLwhVTd0j3m4Mtt")

def scrape_websites(urls: List[str]) -> List[str]:
    logger.info(f"Scraping {len(urls)} websites...")
    run_input = {
        "startUrls": [{"url": url} for url in urls],
        "maxCrawlPages": 10,
        "maxCrawlDepth": 1,
    }
    try:
        run = apify_client.actor("apify/website-content-crawler").call(run_input=run_input)
        dataset_items = apify_client.dataset(run["defaultDatasetId"]).list_items().items
        scraped_content = [item.get('text', '') for item in dataset_items if 'text' in item]
        logger.info(f"Successfully scraped {len(scraped_content)} pages.")
        return scraped_content
    except Exception as e:
        logger.error(f"Error scraping websites: {str(e)}")
        return []

def fetch_scraped_tweets(query: str, max_tweets: int = 100) -> List[Dict[str, Any]]:
    logger.info(f"Fetching tweets for query: {query}")
    actor_input = {
        "queries": [query],
        "maxTweets": max_tweets
    }
    try:
        run = apify_client.actor("apidojo/tweet-scraper").call(run_input=actor_input)
        dataset_id = run["defaultDatasetId"]
        items = apify_client.dataset(dataset_id).list_items().items
        logger.info(f"Fetched {len(items)} tweets.")
        return items
    except Exception as e:
        logger.error(f"Error fetching tweets: {str(e)}")
        return []

def fetch_nvd_data(days: int = 30) -> List[Dict[str, Any]]:
    logger.info(f"Fetching NVD data for the last {days} days...")
    base_url = "https://services.nvd.nist.gov/rest/json/cves/1.0/"
    current_date = datetime.now()
    start_date = current_date - timedelta(days=days)
    params = {
        "pubStartDate": start_date.strftime("%Y-%m-%dT%H:%M:%S:000 UTC-00:00"),
        "pubEndDate": current_date.strftime("%Y-%m-%dT%H:%M:%S:000 UTC-00:00")
    }
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        vulnerabilities = data.get("result", {}).get("CVE_Items", [])
        logger.info(f"Fetched {len(vulnerabilities)} vulnerabilities from NVD.")
        return vulnerabilities
    except Exception as e:
        logger.error(f"Error fetching NVD data: {str(e)}")
        return []

In [7]:
# Expand the list of websites
websites = [
    "https://www.cisa.gov/uscert/ncas/alerts",
    "https://www.virustotal.com/gui/home/upload",
    "https://attack.mitre.org/",
    "https://www.darkreading.com/",
    "https://threatpost.com/",
    "https://krebsonsecurity.com/",
    "https://www.bleepingcomputer.com/",
]

# Data cleaning function
def clean_data(data: List[str]) -> List[str]:
    cleaned_data = []
    for text in data:
        # Remove special characters, HTML tags, etc.
        cleaned_text = re.sub(r'<.*?>', '', text)
        cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_text)
        cleaned_data.append(cleaned_text)
    return cleaned_data

# Function to extract entities and relationships
def extract_entities_and_relationships(text):
    ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")
    entities = ner_pipeline(text)
    for entity in entities:
        entity_type = entity['entity']
        entity_name = entity['word']
        kg.add_entity(entity_name, entity_type)

    # Placeholder for relationship extraction logic
    # You can use dependency parsing or other techniques to extract relationships
    # For simplicity, let's assume we have a function extract_relationships
    relationships = extract_relationships(text)
    for relation in relationships:
        kg.add_relation(relation['entity1'], relation['entity2'], relation['relation'])

# Placeholder function for extracting relationships (needs implementation)
def extract_relationships(text: str) -> List[Dict[str, Any]]:
    # Implement your relationship extraction logic here
    return []

# Scrape websites
scraped_content = scrape_websites(websites)

# Fetch tweets
tweets = fetch_scraped_tweets("#cybersecurity")
tweet_content = [tweet.get('full_text', '') for tweet in tweets]

# Combine scraped content and tweets
all_content = clean_data(scraped_content + tweet_content)

# Populate knowledge graph
for content in all_content:
    extract_entities_and_relationships(content)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.

**Vector Store and Retriever Setup Functions**

In [8]:
def create_vectorstore(texts: List[str]) -> FAISS:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    documents = text_splitter.create_documents(texts)
    return FAISS.from_documents(documents, embeddings)

def setup_retriever(vectorstore: FAISS) -> ContextualCompressionRetriever:
    base_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})
    compressor = LLMChainExtractor.from_llm(llm)
    return ContextualCompressionRetriever(base_compressor=compressor, base_retriever=base_retriever)

# Create vector store and retriever
vectorstore = create_vectorstore(all_content)
retriever = setup_retriever(vectorstore)

**Pydantic Models for Structured Output**

In [9]:
class ThreatAnalysis(BaseModel):
    threat_type: str = Field(description="Type of cybersecurity threat")
    severity: str = Field(description="Severity level of the threat (Low, Medium, High, Critical)")
    description: str = Field(description="Brief description of the threat")
    potential_impact: str = Field(description="Potential impact on organizations")
    mitigation_steps: List[str] = Field(description="List of steps to mitigate the threat")
    ioc_list: Optional[List[str]] = Field(description="List of Indicators of Compromise (IoCs)", default=None)

class VulnerabilityAssessment(BaseModel):
    vulnerability_name: str = Field(description="Name or identifier of the vulnerability")
    affected_systems: List[str] = Field(description="List of affected systems or software")
    cvss_score: float = Field(description="CVSS score of the vulnerability")
    description: str = Field(description="Brief description of the vulnerability")
    remediation_steps: List[str] = Field(description="List of steps to remediate the vulnerability")
    references: Optional[List[str]] = Field(description="List of references for more information", default=None)

class SecurityRecommendation(BaseModel):
    recommendation: str = Field(description="Security recommendation")
    priority: str = Field(description="Priority level (Low, Medium, High)")
    implementation_difficulty: str = Field(description="Difficulty of implementation (Easy, Moderate, Complex)")
    expected_impact: str = Field(description="Expected impact of implementing the recommendation")
    estimated_cost: Optional[str] = Field(description="Estimated cost of implementation", default=None)

**Specialized Agent Tools**

In [10]:
class ThreatAnalyzerTool(BaseTool):
    name = "Threat Analyzer"
    description = "Analyzes cybersecurity threats and provides detailed information"

    def _run(self, query: str) -> ThreatAnalysis:
        # Retrieve relevant information from the vector store
        relevant_docs = retriever.get_relevant_documents(query)
        context = "\n".join([doc.page_content for doc in relevant_docs])

        # Get related entities from the knowledge graph
        related_entities = kg.get_related_entities(query)
        kg_context = "Related entities: {}".format(', '.join([f"{e['entity']} ({e['relation']})" for e in related_entities]))

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="You are a cybersecurity threat analyst. Provide a detailed analysis of the given threat, including its type, severity, description, potential impact, mitigation steps, and if possible, a list of Indicators of Compromise (IoCs). Use the following context:\n\n{}\n\n{}".format(context, kg_context)),
            HumanMessage(content=query)
        ])
        chain = LLMChain(llm=llm, prompt=prompt, output_parser=PydanticOutputParser(pydantic_object=ThreatAnalysis))
        try:
            return chain.run(query)
        except Exception as e:
            logger.error(f"Error in ThreatAnalyzerTool: {str(e)}")
            return ThreatAnalysis(
                threat_type="Unknown",
                severity="Medium",
                description=f"Error analyzing threat: {query}",
                potential_impact="Unknown",
                mitigation_steps=["Consult with cybersecurity experts for a detailed analysis"],
                ioc_list=None
            )

class VulnerabilityAssessorTool(BaseTool):
    name = "Vulnerability Assessor"
    description = "Assesses cybersecurity vulnerabilities and provides detailed information"

    def _run(self, query: str) -> VulnerabilityAssessment:
        # Retrieve relevant information from the vector store
        relevant_docs = retriever.get_relevant_documents(query)
        context = "\n".join([doc.page_content for doc in relevant_docs])

        # Get related entities from the knowledge graph
        related_entities = kg.get_related_entities(query)
        kg_context = "Related entities: {}".format(', '.join([f"{e['entity']} ({e['relation']})" for e in related_entities]))

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="You are a cybersecurity vulnerability assessor. Provide a detailed assessment of the given vulnerability, including its name, affected systems, CVSS score, description, remediation steps, and if available, references for more information. Use the following context:\n\n{}\n\n{}".format(context, kg_context)),
            HumanMessage(content=query)
        ])
        chain = LLMChain(llm=llm, prompt=prompt, output_parser=PydanticOutputParser(pydantic_object=VulnerabilityAssessment))
        try:
            return chain.run(query)
        except Exception as e:
            logger.error(f"Error in VulnerabilityAssessorTool: {str(e)}")
            return VulnerabilityAssessment(
                vulnerability_name="Unknown",
                affected_systems=["Unknown"],
                cvss_score=0.0,
                description=f"Error assessing vulnerability: {query}",
                remediation_steps=["Consult with cybersecurity experts for a detailed assessment"],
                references=None
            )

class SecurityAdvisorTool(BaseTool):
    name = "Security Advisor"
    description = "Provides security recommendations based on the analysis of threats and vulnerabilities"

    def _run(self, query: str) -> SecurityRecommendation:
        # Retrieve relevant information from the vector store
        relevant_docs = retriever.get_relevant_documents(query)
        context = "\n".join([doc.page_content for doc in relevant_docs])

        # Get related entities from the knowledge graph
        related_entities = kg.get_related_entities(query)
        kg_context = "Related entities: {}".format(', '.join([f"{e['entity']} ({e['relation']})" for e in related_entities]))

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="You are a cybersecurity advisor. Provide a security recommendation based on the given analysis, including the recommendation, priority, implementation difficulty, expected impact, and if possible, an estimated cost of implementation. Use the following context:\n\n{}\n\n{}".format(context, kg_context)),
            HumanMessage(content=query)
        ])
        chain = LLMChain(llm=llm, prompt=prompt, output_parser=PydanticOutputParser(pydantic_object=SecurityRecommendation))
        try:
            return chain.run(query)
        except Exception as e:
            logger.error(f"Error in SecurityAdvisorTool: {str(e)}")
            return SecurityRecommendation(
                recommendation="Unable to provide specific recommendation",
                priority="Medium",
                implementation_difficulty="Unknown",
                expected_impact="Unknown",
                estimated_cost=None
            )

In [12]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.tools import BaseTool
from langchain.output_parsers import PydanticOutputParser
from typing import List, Dict, Any

tools = [
    ThreatAnalyzerTool(),
    VulnerabilityAssessorTool(),
    SecurityAdvisorTool(),
]

template = """You are a cybersecurity agent. Use the following tools to provide detailed analyses and recommendations based on the input: {tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
{agent_scratchpad}"""

prompt = ChatPromptTemplate.from_template(template)

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Create an agent executor
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

**Hierarchical Agents Team**

In [13]:
def process_queries(queries: List[str]) -> List[str]:
    results = []
    for query in queries:
        try:
            result = agent_executor.invoke(
                {
                    "input": query,
                }
            )
            results.append(result["output"])
        except ValueError as e:
            logger.error(f"Error processing query '{query}': {str(e)}")
            results.append(f"Error: Unable to process query due to an internal error. Please try rephrasing or simplifying your query.")
        except Exception as e:
            logger.error(f"Unexpected error processing query '{query}': {str(e)}")
            results.append(f"Error: An unexpected error occurred. Please try again later.")
    return results

# Main execution
if __name__ == "__main__":
    # Example usage
    queries = [
        "Assess the vulnerability CVE-2024-12345 in Windows Server.",
        "Provide a security recommendation for mitigating phishing attacks.",
        "List all details on BFSI security incidents in India.",
        "List all ransomware attacks targeting the healthcare industry in the last 7 days.",
        "Provide recent incidents related to Lockbit Ransomware gang.",
        "Provide recent incidents related to BlackBasta Ransomware."
    ]

    results = process_queries(queries)
    for i, result in enumerate(results):
        print(f"Result for Query {i+1}: {result}")

    print("Cybersecurity co-pilot pipeline executed successfully.")



> Entering new AgentExecutor chain...
Thought: To assess the vulnerability CVE-2024-12345 in Windows Server, I need to gather more information about the vulnerability itself.

Action: Vulnerability Assessor
Action Input: CVE-2024-12345

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
ERROR:__main__:Unexpected error processing query 'Assess the vulnerability CVE-2024-12345 in Windows Server.': The node CVE-2024-12345 is not in the graph.




> Entering new AgentExecutor chain...
Thought: To provide a security recommendation for mitigating phishing attacks, I need to understand the nature of phishing threats and the vulnerabilities that attackers often exploit.

Action: Threat Analyzer
Action Input: Phishing attacks

ERROR:__main__:Unexpected error processing query 'Provide a security recommendation for mitigating phishing attacks.': The node Phishing attacks is not in the graph.




> Entering new AgentExecutor chain...
Thought: To provide detailed information on BFSI (Banking, Financial Services, and Insurance) security incidents in India, I need to analyze the threats and vulnerabilities in the BFSI sector.

Action: Threat Analyzer
Action Input: BFSI security incidents in India

...



> Entering new AgentExecutor chain...
Thought: To provide recent incidents related to Lockbit Ransomware gang, I need to analyze the latest threats and gather information on the gang's activities.

Action: Threat Analyzer
Action Input: Lockbit Ransomware gang, recent incidents

ERROR:__main__:Unexpected error processing query 'Provide recent incidents related to Lockbit Ransomware gang.': The node Lockbit Ransomware gang, recent incidents is not in the graph.




> Entering new AgentExecutor chain...
Thought: To provide recent incidents related to BlackBasta Ransomware, I need to analyze the threat landscape and gather information on recent attacks.

Action: Threat Analyzer
Action Input: BlackBasta Ransomware, recent incidents

ERROR:__main__:Unexpected error processing query 'Provide recent incidents related to BlackBasta Ransomware.': The node BlackBasta Ransomware, recent incidents is not in the graph.


Result for Query 1: Error: An unexpected error occurred. Please try again later.
Result for Query 2: Error: An unexpected error occurred. Please try again later.
Result for Query 3: Error: An unexpected error occurred. Please try again later.
Result for Query 4: Error: An unexpected error occurred. Please try again later.
Result for Query 5: Error: An unexpected error occurred. Please try again later.
Result for Query 6: Error: An unexpected error occurred. Please try again later.
Cybersecurity co-pilot pipeline executed successfully.
